# Andhra_Bus_details

In [2]:
#importing all required libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of APSRTC state from Redbus.in
URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

# Define a function to open the browser
def run_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver 

# To fetch the webpage and wait
def access_page(driver, url):
    driver.get(url)
    time.sleep(3)

# To scrape bus_route_name and bus_route_link
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href') is not None]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details and allow the page to load
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  

        # Click the "View Buses" button if it exists and wait until the buses load
        try:
            view_buses_button = driver.find_element(By.CLASS_NAME, "button")
            actions = ActionChains(driver)
            actions.move_to_element(view_buses_button).click().perform()
            time.sleep(5)
            
            # Scroll down to load all bus items
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(10)  # Adjust the sleep time if necessary
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height


            # Locate the multiple bus elements 
            bus_name_elements = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            # Scrape bus details from a webpage and store them in a list of dictionaries 
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seats_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# Initialize a list to store all scraped data from multiple pages
all_bus_details = []

# Defining a function to navigate pagination and scrape all bus details from multiple pages
def scrape_all_pages():
    for page in range(1, 6): 
        try:
            driver = run_driver()
            access_page(driver, URL)

            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")


    driver.quit()  


# Scrape routes and details from all pages
scrape_all_pages()

# Convert into a DataFrame
df1 = pd.DataFrame(all_bus_details)

# Saving the DataFrame to a CSV file
df1.to_csv('Andhra_bus_details.csv', index=False)


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/vinukonda-to-hyderabad: Message: no such element: Unable to locate element: {"method":"css selector","selector":".button"}
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF74B4E9412+29090]
	(No symbol) [0x00007FF74B45E239]
	(No symbol) [0x00007FF74B31B1DA]
	(No symbol) [0x00007FF74B36EFE7]
	(No symbol) [0x00007FF74B36F23C]
	(No symbol) [0x00007FF74B3B97C7]
	(No symbol) [0x00007FF74B39672F]
	(No symbol) [0x00007FF74B3B65A2]
	(No symbol) [0x00007FF74B396493]
	(No symbol) [0x00007FF74B3609D1]
	(No symbol) [0x00007FF74B361B31]
	GetHandleVerifier [0x00007FF74B80871D+3302573]
	GetHandleVerifier [0x00007FF74B854243+3612627]
	GetHandleVerifier [0x00007FF74B84A417+3572135]
	GetHandleVerifier [0x00007FF74B5A5EB6+801862]
	(No symbol) [0x00007F

In [4]:
df1

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seats_Available
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3977,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",20:15,08h 00m,04:15,3.5,INR 469,16 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 2876,STAR LINER(NON-AC SLEEPER 2+1),20:15,07h 40m,03:55,3.9,INR 565,23 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 2549,NIGHT RIDER (SEATER CUM SLEEPER),20:15,06h 00m,02:15,3.1,INR 595,29 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 37801,INDRA(A.C. Seater),20:20,07h 25m,03:45,3.1,INR 567,24 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 24262,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",20:30,05h 30m,02:00,3.9,INR 469,20 Seats available
...,...,...,...,...,...,...,...,...,...,...
3678,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,True Bus,Volvo Multi Axle B9R A/C Sleeper (2+1),17:00,06h 59m,23:59,3.0,INR 4400,36 Seats available
3679,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,Muneer Travels,Non A/C Seater / Sleeper (2+1),22:15,05h 30m,03:45,2.2,INR 500,40 Seats available
3680,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,DHANUNJAYA TRAVELS,A/C Sleeper (2+1),21:00,06h 00m,03:00,1.8,INR 1000,36 Seats available
3681,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,Meghana Travels,A/C Sleeper (2+1),22:20,05h 55m,04:15,1.7,2850,9 Seats available


# Telangana_Bus_details

In [7]:
#importing all required libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of APSRTC state from Redbus.in
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

# Define a function to open the browser
def run_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver 

# To fetch the webpage and wait
def access_page(driver, url):
    driver.get(url)
    time.sleep(3)

# To scrape bus_route_name and bus_route_link
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href') is not None]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details and allow the page to load
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  

        # Click the "View Buses" button if it exists and wait until the buses load
        try:
            view_buses_button = driver.find_element(By.CLASS_NAME, "button")
            actions = ActionChains(driver)
            actions.move_to_element(view_buses_button).click().perform()
            time.sleep(5)
            
            # Scroll down to load all bus items
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(10)  # Adjust the sleep time if necessary
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height


            # Locate the multiple bus elements 
            bus_name_elements = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            # Scrape bus details from a webpage and store them in a list of dictionaries 
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seats_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# Initialize a list to store all scraped data from multiple pages
all_bus_details = []

# Defining a function to navigate pagination and scrape all bus details from multiple pages
def scrape_all_pages():
    for page in range(1, 4): 
        try:
            driver = run_driver()
            access_page(driver, URL)

            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")


    driver.quit()  


# Scrape routes and details from all pages
scrape_all_pages()

# Convert into a DataFrame
df2 = pd.DataFrame(all_bus_details)

# Saving the DataFrame to a CSV file
df2.to_csv('telangana_bus_details.csv', index=False)


In [9]:
df2

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seats_Available
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 351880,VENNELA (A.C. SLEEPER),00:40,05h 25m,06:05,4.7,INR 737,16 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3563,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:00,06h 15m,07:15,3.8,INR 412,33 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35266,VENNELA (A.C. SLEEPER),01:25,05h 24m,06:49,3.9,INR 737,27 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3590,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:45,06h 15m,08:00,4.5,INR 412,33 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3058,VENNELA (A.C. SLEEPER),02:20,05h 20m,07:40,2.9,INR 781,13 Seats available
...,...,...,...,...,...,...,...,...,...,...
2321,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,KKaveri Travels,A/C Sleeper (2+1),22:00,06h 30m,04:30,1.6,INR 1000,28 Seats available
2322,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,KKaveri Travels,A/C Sleeper (2+1),21:00,07h 30m,04:30,1.3,INR 900,28 Seats available
2323,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Sai Durga Travels,Non A/C Seater / Sleeper (2+1),23:45,07h 15m,07:00,0,INR 5000,1 Seat available
2324,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Star Bus,Volvo Multi-Axle A/C Seater/Sleeper (2+1),23:30,09h 15m,08:45,0,1425,47 Seats available


# Kerala _Bus_details

In [12]:
#importing all required libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of website
URL = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"

# Define a function to open the browser
def run_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver 

# To fetch the webpage and wait
def access_page(driver, url):
    driver.get(url)
    time.sleep(3)

# To scrape bus_route_name and bus_route_link
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href') is not None]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details and allow the page to load
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  

        # Click the "View Buses" button if it exists and wait until the buses load
        try:
            view_buses_button = driver.find_element(By.CLASS_NAME, "button")
            actions = ActionChains(driver)
            actions.move_to_element(view_buses_button).click().perform()
            time.sleep(5)
            
            # Scroll down to load all bus items
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(10)  # Adjust the sleep time if necessary
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height


            # Locate the multiple bus elements 
            bus_name_elements = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            # Scrape bus details from a webpage and store them in a list of dictionaries 
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seats_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# Initialize a list to store all scraped data from multiple pages
all_bus_details = []

# Defining a function to navigate pagination and scrape all bus details from multiple pages
def scrape_all_pages():
    for page in range(1, 3): 
        try:
            driver = run_driver()
            access_page(driver, URL)

            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")


    driver.quit()  


# Scrape routes and details from all pages
scrape_all_pages()

# Convert into a DataFrame
df3 = pd.DataFrame(all_bus_details)

# Saving the DataFrame to a CSV file
df3.to_csv('kerala_bus_details.csv', index=False)


In [14]:
df3

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seats_Available
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),10:03,08h 27m,18:30,4.1,INR 606,16 Seats available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:01,07h 54m,19:55,3.5,INR 945,16 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,07h 39m,21:25,2.0,INR 1099,32 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,08h 35m,22:50,4.2,INR 945,33 Seats available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,09h 01m,00:01,3.9,INR 801,24 Seats available
...,...,...,...,...,...,...,...,...,...,...
721,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Madhavi Travels,Volvo A/C Semi Sleeper (2+2),23:59,04h 31m,04:30,3.7,INR 1999,32 Seats available
722,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Ubc travels,A/C Semi Sleeper (2+2),17:30,07h 30m,01:00,3.2,INR 900,24 Seats available
723,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,EMERALD TRAVELS,A/C Sleeper (2+1),23:00,05h 30m,04:30,3.0,INR 2499,7 Seats available
724,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Jacobs Travels,A/C Semi Sleeper (2+2),18:30,08h 00m,02:30,2.7,INR 950,14 Seats available


# kadamba_Bus_details

In [17]:
#importing all required libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of website
URL = "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"

# Define a function to open the browser
def run_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver 

# To fetch the webpage and wait
def access_page(driver, url):
    driver.get(url)
    time.sleep(3)

# To scrape bus_route_name and bus_route_link
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href') is not None]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details and allow the page to load
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  

        # Click the "View Buses" button if it exists and wait until the buses load
        try:
            view_buses_button = driver.find_element(By.CLASS_NAME, "button")
            actions = ActionChains(driver)
            actions.move_to_element(view_buses_button).click().perform()
            time.sleep(5)
            
            # Scroll down to load all bus items
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(10)  # Adjust the sleep time if necessary
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height


            # Locate the multiple bus elements 
            bus_name_elements = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            # Scrape bus details from a webpage and store them in a list of dictionaries 
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seats_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# Initialize a list to store all scraped data from multiple pages
all_bus_details = []

# Defining a function to navigate pagination and scrape all bus details from multiple pages
def scrape_all_pages():
    for page in range(1, 5): 
        try:
            driver = run_driver()
            access_page(driver, URL)

            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")


    driver.quit()  


# Scrape routes and details from all pages
scrape_all_pages()

# Convert into a DataFrame
df4 = pd.DataFrame(all_bus_details)

# Saving the DataFrame to a CSV file
df4.to_csv('kadamba_bus_details.csv', index=False)


In [19]:
df4

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seats_Available
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:15,10h 35m,05:50,3.8,INR 900,17 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:30,10h 00m,07:30,4.6,INR 599,18 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),23:00,09h 45m,08:45,4.5,INR 599,25 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,AdIntrCity SmartBus,AC Sleeper (2+1),22:00,10h 15m,08:15,3.7,INR 599,23 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:35,10h 55m,08:30,4.3,INR 749,13 Seats available
...,...,...,...,...,...,...,...,...,...,...
518,Calangute (goa) to Goa,https://www.redbus.in/bus-tickets/calangute-go...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,00h 31m,16:01,3.7,INR 100,39 Seats available
519,Calangute (goa) to Goa,https://www.redbus.in/bus-tickets/calangute-go...,UNIQUE TOURS AND TRAVELS,A/C Seater Push Back (2+1),08:30,11h 30m,20:00,3.2,INR 400,17 Seats available
520,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),03:30,04h 30m,08:00,1.3,INR 1550,17 Seats available
521,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa,Shirdi Xpress,A/C Sleeper (2+1),08:15,03h 15m,11:30,2.2,INR 419,16 Seats available


# Rajasthan_Bus_details

In [22]:
#importing all required libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of website
URL = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"

# Define a function to open the browser
def run_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver 

# To fetch the webpage and wait
def access_page(driver, url):
    driver.get(url)
    time.sleep(3)

# To scrape bus_route_name and bus_route_link
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href') is not None]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details and allow the page to load
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  

        # Click the "View Buses" button if it exists and wait until the buses load
        try:
            view_buses_button = driver.find_element(By.CLASS_NAME, "button")
            actions = ActionChains(driver)
            actions.move_to_element(view_buses_button).click().perform()
            time.sleep(5)
            
            # Scroll down to load all bus items
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(10)  # Adjust the sleep time if necessary
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height


            # Locate the multiple bus elements 
            bus_name_elements = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            # Scrape bus details from a webpage and store them in a list of dictionaries 
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seats_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# Initialize a list to store all scraped data from multiple pages
all_bus_details = []

# Defining a function to navigate pagination and scrape all bus details from multiple pages
def scrape_all_pages():
    for page in range(1, 3): 
        try:
            driver = run_driver()
            access_page(driver, URL)

            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")


    driver.quit()  


# Scrape routes and details from all pages
scrape_all_pages()

# Convert into a DataFrame
df5 = pd.DataFrame(all_bus_details)

# Saving the DataFrame to a CSV file
df5.to_csv('rajasthan_bus_details.csv', index=False)


In [24]:
df5

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seats_Available
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 174534,Express Non AC Seater 2+3,05:45,06h 15m,12:00,4.2,INR 299,41 Seats available
1,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 158798,Super Luxury Volvo AC Seater Pushback 2+2,06:01,04h 59m,11:00,4.3,INR 602,1 Seat available
2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 146395,Express Non AC Seater 2+3,06:45,07h 15m,14:00,4.2,INR 299,42 Seats available
3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 174533,Express Non AC Seater 2+3,08:45,07h 15m,16:00,4.2,INR 299,42 Seats available
4,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 188347,Super Luxury Volvo AC Seater Pushback 2+2,11:30,05h 00m,16:30,4.2,INR 602,13 Seats available
...,...,...,...,...,...,...,...,...,...,...
1644,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Shree Hare Rama Travels,A/C Seater/Sleeper (2+1),06:00,04h 30m,10:30,2.6,2279,29 Seats available
1645,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Geetanjali Travels,NON AC Seater / Sleeper 2+1,23:00,05h 30m,04:30,2.5,INR 300,40 Seats available
1646,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,New shiv mahima travels express & cargo,NON A/C Sleeper (2+1),19:15,04h 50m,00:05,1.8,INR 650,30 Seats available
1647,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,A One Travel Agency,Non A/C Seater / Sleeper (2+1),21:00,05h 30m,02:30,1.6,INR 350,27 Seats available


# SouthBengal_Bus_details

In [1]:
#importing all required libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of website
URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"

# Define a function to open the browser
def run_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver 

# To fetch the webpage and wait
def access_page(driver, url):
    driver.get(url)
    time.sleep(3)

# To scrape bus_route_name and bus_route_link
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href') is not None]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details and allow the page to load
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  

        # Click the "View Buses" button if it exists and wait until the buses load
        try:
            view_buses_button = driver.find_element(By.CLASS_NAME, "button")
            actions = ActionChains(driver)
            actions.move_to_element(view_buses_button).click().perform()
            time.sleep(5)
            
            # Scroll down to load all bus items
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(10)  # Adjust the sleep time if necessary
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height


            # Locate the multiple bus elements 
            bus_name_elements = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            # Scrape bus details from a webpage and store them in a list of dictionaries 
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seats_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# Initialize a list to store all scraped data from multiple pages
all_bus_details = []

# Defining a function to navigate pagination and scrape all bus details from multiple pages
def scrape_all_pages():
    for page in range(1, 6): 
        try:
            driver = run_driver()
            access_page(driver, URL)

            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")


    driver.quit()  


# Scrape routes and details from all pages
scrape_all_pages()

# Convert into a DataFrame
df6 = pd.DataFrame(all_bus_details)

# Saving the DataFrame to a CSV file
df6.to_csv('southbengal_bus_details.csv', index=False)


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-midnapore: Message: no such element: Unable to locate element: {"method":"css selector","selector":".button"}
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60F589412+29090]
	(No symbol) [0x00007FF60F4FE239]
	(No symbol) [0x00007FF60F3BB1DA]
	(No symbol) [0x00007FF60F40EFE7]
	(No symbol) [0x00007FF60F40F23C]
	(No symbol) [0x00007FF60F4597C7]
	(No symbol) [0x00007FF60F43672F]
	(No symbol) [0x00007FF60F4565A2]
	(No symbol) [0x00007FF60F436493]
	(No symbol) [0x00007FF60F4009D1]
	(No symbol) [0x00007FF60F401B31]
	GetHandleVerifier [0x00007FF60F8A871D+3302573]
	GetHandleVerifier [0x00007FF60F8F4243+3612627]
	GetHandleVerifier [0x00007FF60F8EA417+3572135]
	GetHandleVerifier [0x00007FF60F645EB6+801862]
	(No symbol)

In [3]:
df6

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seats_Available
0,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KOLKATA - 12:00 (DGP DEPOT) -...,Non AC Seater (2+3),12:00,03h 30m,15:30,3.9,INR 150,52 Seats available
1,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-BANKURA - KARUNAMOYEE - 10:15 (BNK DEPOT...,Non AC Seater (2+3),12:15,04h 00m,16:15,3.9,INR 155,52 Seats available
2,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KOLKATA - L/S - 11:10 (ASANSOL...,Non AC Seater (2+3),12:20,03h 30m,15:50,2.3,INR 150,41 Seats available
3,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KOLKATA - 11:30 (HOWRAH DEPOT)...,Non AC Seater (2+3),12:40,03h 30m,16:10,4.6,INR 150,50 Seats available
4,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KARUNAMOYEE - 11:35 (BEL DEPOT...,Non AC Seater (2+3),12:45,04h 00m,16:45,3.7,INR 155,52 Seats available
...,...,...,...,...,...,...,...,...,...,...
550,Kolkata to Ashari,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - JHARGRAM - VIA - LODHASULI - 1...,Non AC Seater (2+3),16:20,07h 40m,00:00,4.1,INR 82,52 Seats available
551,Kolkata to Ashari,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - MIDNAPUR - 18:00 (MID DEPOT) -...,Non AC Seater (2+3),18:00,06h 00m,00:00,4.3,INR 82,50 Seats available
552,Kolkata to Ashari,https://www.redbus.in/bus-tickets/kolkata-to-a...,SBSTC-KOLKATA - MIDNAPUR - 18:45 (MID DEPOT) -...,Non AC Seater (2+3),18:45,05h 15m,00:00,3.5,INR 82,47 Seats available
553,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) HABRA-DIGHA via Bally - 26|14:00,Non AC Seater (2+3),14:00,04h 00m,18:00,3.3,INR 161,37 Seats available


# Himachal-Pradesh_Bus_details

In [6]:
#importing all required libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of website
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"

# Define a function to open the browser
def run_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver 

# To fetch the webpage and wait
def access_page(driver, url):
    driver.get(url)
    time.sleep(3)

# To scrape bus_route_name and bus_route_link
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href') is not None]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details and allow the page to load
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  

        # Click the "View Buses" button if it exists and wait until the buses load
        try:
            view_buses_button = driver.find_element(By.CLASS_NAME, "button")
            actions = ActionChains(driver)
            actions.move_to_element(view_buses_button).click().perform()
            time.sleep(5)
            
            # Scroll down to load all bus items
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(10)  # Adjust the sleep time if necessary
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height


            # Locate the multiple bus elements 
            bus_name_elements = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            # Scrape bus details from a webpage and store them in a list of dictionaries 
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seats_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# Initialize a list to store all scraped data from multiple pages
all_bus_details = []

# Defining a function to navigate pagination and scrape all bus details from multiple pages
def scrape_all_pages():
    for page in range(1, 5): 
        try:
            driver = run_driver()
            access_page(driver, URL)

            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")


    driver.quit()  


# Scrape routes and details from all pages
scrape_all_pages()

# Convert into a DataFrame
df7 = pd.DataFrame(all_bus_details)

# Saving the DataFrame to a CSV file
df7.to_csv('himachalpradesh_bus_details.csv', index=False)


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-baddi-himachal-pradesh: Message: no such element: Unable to locate element: {"method":"css selector","selector":".button"}
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60F589412+29090]
	(No symbol) [0x00007FF60F4FE239]
	(No symbol) [0x00007FF60F3BB1DA]
	(No symbol) [0x00007FF60F40EFE7]
	(No symbol) [0x00007FF60F40F23C]
	(No symbol) [0x00007FF60F4597C7]
	(No symbol) [0x00007FF60F43672F]
	(No symbol) [0x00007FF60F4565A2]
	(No symbol) [0x00007FF60F436493]
	(No symbol) [0x00007FF60F4009D1]
	(No symbol) [0x00007FF60F401B31]
	GetHandleVerifier [0x00007FF60F8A871D+3302573]
	GetHandleVerifier [0x00007FF60F8F4243+3612627]
	GetHandleVerifier [0x00007FF60F8EA417+3572135]
	GetHandleVerifier [0x00007FF60F645EB6+801862]
	(No symbol) 

In [8]:
df7

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seats_Available
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 551,A/C Executive (2+3),15:30,10h 00m,01:30,3.5,INR 632,42 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 525,Ordinary 3+2 Non AC Seater,16:50,10h 10m,03:00,4.2,INR 512,27 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 532,Ordinary 3+2 Non AC Seater,17:59,10h 16m,04:15,2.5,INR 512,27 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 538,Ordinary 3+2 Non AC Seater,18:31,10h 59m,05:30,4.6,INR 512,24 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,09h 10m,04:45,3.6,INR 912,28 Seats available
...,...,...,...,...,...,...,...,...,...,...
902,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Rao Travel Heights,Volvo A/C Semi Sleeper (2+2),18:30,12h 30m,07:00,0,1443,41 Seats available
903,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Yatrabybus.com,Bharat Benz A/C Semi Sleeper (2+2),18:00,10h 00m,04:00,0,INR 1199,46 Seats available
904,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Royal Travels Hills,Scania Multi-Axle AC Semi Sleeper (2+2),21:00,10h 30m,07:30,0,INR 1099,49 Seats available
905,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi,Travel hub,Volvo A/C Semi Sleeper (2+2),19:30,12h 00m,07:30,0,INR 1099,27 Seats available


# Uttar-Pradesh_Bus_details

In [22]:
#importing all required libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

# Define a function to open the browser
def run_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver 

# To fetch the webpage and wait
def access_page(driver, url):
    driver.get(url)
    time.sleep(3)

# To scrape bus_route_name and bus_route_link
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href') is not None]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details and allow the page to load
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  

        # Click the "View Buses" button if it exists and wait until the buses load
        try:
            view_buses_button = driver.find_element(By.CLASS_NAME, "button")
            actions = ActionChains(driver)
            actions.move_to_element(view_buses_button).click().perform()
            time.sleep(5)
            
            # Scroll down to load all bus items
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(10)  # Adjust the sleep time if necessary
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height


            # Locate the multiple bus elements 
            bus_name_elements = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            # Scrape bus details from a webpage and store them in a list of dictionaries 
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seats_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# Initialize a list to store all scraped data from multiple pages
all_bus_details = []

# Defining a function to navigate pagination and scrape all bus details from multiple pages
def scrape_all_pages():
    for page in range(1, 6): 
        try:
            driver = run_driver()
            access_page(driver, URL)

            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")


    driver.quit()  


# Scrape routes and details from all pages
scrape_all_pages()

# Convert into a DataFrame
df8 = pd.DataFrame(all_bus_details)

# Saving the DataFrame to a CSV file
df8.to_csv('uttarpradesh_bus_details.csv', index=False)


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/lucknow-to-aligarh: Message: no such element: Unable to locate element: {"method":"css selector","selector":".button"}
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60F589412+29090]
	(No symbol) [0x00007FF60F4FE239]
	(No symbol) [0x00007FF60F3BB1DA]
	(No symbol) [0x00007FF60F40EFE7]
	(No symbol) [0x00007FF60F40F23C]
	(No symbol) [0x00007FF60F4597C7]
	(No symbol) [0x00007FF60F43672F]
	(No symbol) [0x00007FF60F4565A2]
	(No symbol) [0x00007FF60F436493]
	(No symbol) [0x00007FF60F4009D1]
	(No symbol) [0x00007FF60F401B31]
	GetHandleVerifier [0x00007FF60F8A871D+3302573]
	GetHandleVerifier [0x00007FF60F8F4243+3612627]
	GetHandleVerifier [0x00007FF60F8EA417+3572135]
	GetHandleVerifier [0x00007FF60F645EB6+801862]
	(No symbol) [0x00007FF60F

In [24]:
df8

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seats_Available
0,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - BDN0037,Ordinary Non AC Seater 2+3,14:00,07h 30m,21:30,3.3,INR 476,52 Seats available
1,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - HPR0010,Ordinary Non AC Seater 2+3,14:00,06h 30m,20:30,3.3,INR 444,52 Seats available
2,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - BRH0245,Ordinary Non AC Seater 2+3,14:00,05h 50m,19:50,3.3,INR 426,52 Seats available
3,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - BLP0029,Ordinary Non AC Seater 2+3,14:01,05h 28m,19:29,3.3,INR 426,52 Seats available
4,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - RKD0151,Ordinary Non AC Seater 2+3,14:01,05h 30m,19:31,3.3,INR 415,52 Seats available
...,...,...,...,...,...,...,...,...,...,...
2158,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - MNP0132,Ordinary Non AC Seater 2+3,23:32,04h 40m,04:12,2.3,INR 371,52 Seats available
2159,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,Chaturvedi Bus Service,NON AC Seater / Sleeper 2+1,20:30,06h 00m,02:30,2.3,INR 620,45 Seats available
2160,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,Chaturvedi Bus Service,NON A/C Seater/ Sleeper (2+1),21:00,06h 30m,03:30,2.3,INR 570,51 Seats available
2161,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,Chaturvedi Bus Service,NON A/C Seater/ Sleeper (2+1),22:45,06h 30m,05:15,2.3,INR 450,51 Seats available


# Chandigarh_Bus_details

In [32]:
#importing all required libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of website
URL = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"

# Define a function to open the browser
def run_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver 

# To fetch the webpage and wait
def access_page(driver, url):
    driver.get(url)
    time.sleep(3)

# To scrape bus_route_name and bus_route_link
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href') is not None]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details and allow the page to load
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  

        # Click the "View Buses" button if it exists and wait until the buses load
        try:
            view_buses_button = driver.find_element(By.CLASS_NAME, "button")
            actions = ActionChains(driver)
            actions.move_to_element(view_buses_button).click().perform()
            time.sleep(5)
            
            # Scroll down to load all bus items
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(10)  # Adjust the sleep time if necessary
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height


            # Locate the multiple bus elements 
            bus_name_elements = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            # Scrape bus details from a webpage and store them in a list of dictionaries 
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seats_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# Initialize a list to store all scraped data from multiple pages
all_bus_details = []

# Defining a function to navigate pagination and scrape all bus details from multiple pages
def scrape_all_pages():
    for page in range(1, 6): 
        try:
            driver = run_driver()
            access_page(driver, URL)

            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")


    driver.quit()  


# Scrape routes and details from all pages
scrape_all_pages()

# Convert into a DataFrame
df9= pd.DataFrame(all_bus_details)

# Saving the DataFrame to a CSV file
df9.to_csv('chandigarh_bus_details.csv', index=False)


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/talwara-to-chandigarh: Message: no such element: Unable to locate element: {"method":"css selector","selector":".button"}
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60F589412+29090]
	(No symbol) [0x00007FF60F4FE239]
	(No symbol) [0x00007FF60F3BB1DA]
	(No symbol) [0x00007FF60F40EFE7]
	(No symbol) [0x00007FF60F40F23C]
	(No symbol) [0x00007FF60F4597C7]
	(No symbol) [0x00007FF60F43672F]
	(No symbol) [0x00007FF60F4565A2]
	(No symbol) [0x00007FF60F436493]
	(No symbol) [0x00007FF60F4009D1]
	(No symbol) [0x00007FF60F401B31]
	GetHandleVerifier [0x00007FF60F8A871D+3302573]
	GetHandleVerifier [0x00007FF60F8F4243+3612627]
	GetHandleVerifier [0x00007FF60F8EA417+3572135]
	GetHandleVerifier [0x00007FF60F645EB6+801862]
	(No symbol) [0x00007FF

In [34]:
df9

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seats_Available
0,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Zimindara Travels,AC Sleeper (2+1),20:30,04h 00m,00:30,4.0,INR 674,29 Seats available
1,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),VE A/C Seater / Sleeper (2+1),20:40,02h 50m,23:30,3.7,INR 680,22 Seats available
2,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,City Land Travels,A/C Seater/Sleeper (2+1),22:20,02h 00m,00:20,3.5,601,27 Seats available
3,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),21:00,02h 30m,23:30,3.5,554,38 Seats available
4,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165720,HVAC Seater (2+3),15:30,05h 35m,21:05,3.7,INR 414.3,47 Seats available
...,...,...,...,...,...,...,...,...,...,...
720,Chandigarh to Haldwani,https://www.redbus.in/bus-tickets/chandigarh-t...,Shree Mahaveer Travels Jaipur,A/C Seater/Sleeper (2+1),21:00,09h 55m,06:55,3.5,664,19 Seats available
721,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),15:25,04h 05m,19:30,3.6,INR 650,38 Seats available
722,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),17:25,03h 55m,21:20,4.3,INR 650,43 Seats available
723,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),14:55,04h 05m,19:00,2.7,INR 666,38 Seats available


# Punjab_Bus_details

In [38]:
#importing all required libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of website
URL = "https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile"

# Define a function to open the browser
def run_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver 

# To fetch the webpage and wait
def access_page(driver, url):
    driver.get(url)
    time.sleep(3)

# To scrape bus_route_name and bus_route_link
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements if route.get_attribute('href') is not None]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details and allow the page to load
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  

        # Click the "View Buses" button if it exists and wait until the buses load
        try:
            view_buses_button = driver.find_element(By.CLASS_NAME, "button")
            actions = ActionChains(driver)
            actions.move_to_element(view_buses_button).click().perform()
            time.sleep(5)
            
            # Scroll down to load all bus items
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(10)  # Adjust the sleep time if necessary
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height


            # Locate the multiple bus elements 
            bus_name_elements = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            # Scrape bus details from a webpage and store them in a list of dictionaries 
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seats_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# Initialize a list to store all scraped data from multiple pages
all_bus_details = []

# Defining a function to navigate pagination and scrape all bus details from multiple pages
def scrape_all_pages():
    for page in range(1, 3): 
        try:
            driver = run_driver()
            access_page(driver, URL)

            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")


    driver.quit()  


# Scrape routes and details from all pages
scrape_all_pages()

# Convert into a DataFrame
df10 = pd.DataFrame(all_bus_details)

# Saving the DataFrame to a CSV file
df10.to_csv('punjab_bus_deta.csv', index=False)


In [39]:
df10

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seats_Available
0,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 66667,Volvo AC Seater (2+2),17:30,05h 00m,22:30,4.4,INR 711,39 Seats available
1,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 67103,AC Seater Hvac 2+2,22:30,05h 30m,04:00,3.3,INR 454,36 Seats available
2,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 66687,Volvo AC Seater (2+2),18:10,06h 09m,00:19,3.2,INR 711,36 Seats available
3,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 67104,AC Seater Hvac 2+2,22:30,05h 01m,03:31,3.1,INR 454,40 Seats available
4,Delhi to Ludhiana,https://www.redbus.in/bus-tickets/delhi-to-lud...,PEPSU (Punjab) - 67166,Volvo AC Seater (2+2),16:50,06h 00m,22:50,3.7,INR 873,39 Seats available
...,...,...,...,...,...,...,...,...,...,...
410,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),15:30,11h 15m,02:45,4.6,INR 2500,51 Seats available
411,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),23:30,11h 15m,10:45,4.6,INR 2500,44 Seats available
412,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),19:30,09h 15m,04:45,4.6,INR 2500,51 Seats available
413,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),22:00,09h 15m,07:15,4.6,INR 3000,42 Seats available
